In [ ]:
import matplotlib.pyplot as plt
import scipy.constants
import numpy as np
from scipy.optimize import curve_fit
from scipy.interpolate import UnivariateSpline

from pmd_beamphysics.wavefront import Wavefront

In [ ]:
%config InlineBackend.figure_format = 'retina'

## Creating a Wavefront

In [ ]:
W = Wavefront.gaussian_pulse(
    dims=(101, 101, 801),
    wavelength=1.35e-8,
    grid_spacing=(6e-6, 6e-6, 1e-7),
    pad=(100, 100, 40),
    nphotons=1e12,
    zR=2.0,
    sigma_z=3e-6,
)

In [ ]:
W.plot("im", projection="xy", figsize=(3, 3));

In [ ]:
W.plot("re", projection="xy", figsize=(3, 3));

In [ ]:
W.plot("re", projection="kxky", xlim=(-0.1, 0.1), ylim=(-0.1, 0.1), figsize=(3, 3));

In [ ]:
W.fft_unit_coeff

In [ ]:
W.plot(
    "phase",
    projection="kxky",
    xlim=(-0.1, 0.1),
    ylim=(-0.1, 0.1),
    isophase_contour=True,
);

In [ ]:
W.plot_1d_far_field_spectral_density();

In [ ]:
W.plot_reciprocal(ylim_theta=(-0.5, 0.5));

In [ ]:
# W.plot_1d_kmesh_projections();

In [ ]:
def get_longitudinal_slice(arr):
    nx, ny, _ = arr.shape
    return arr[int(nx / 2), int(ny / 2), :]


def get_transverse_slice(arr):
    _, _, nz = arr.shape
    return arr[:, :, int(nz / 2)]

In [ ]:
plt.figure(figsize=(4, 2))
plt.plot(W.rspace_domain[2], np.abs(get_longitudinal_slice(W.rmesh)))
plt.xlabel("Position z [m]");

In [ ]:
plt.figure(figsize=(4, 2))
plt.plot(
    W.rspace_domain[2] / scipy.constants.c * 1e15,
    np.abs(get_longitudinal_slice(W.rmesh)),
)
plt.xlabel("Time [fs]");

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(np.abs(get_transverse_slice(W.rmesh) ** 2))
plt.colorbar();

In [ ]:
plt.figure(figsize=(4, 2))
kspace_slice = np.abs(get_longitudinal_slice(W.kmesh))
plt.plot(kspace_slice, ".-", color="b")
peak_x = np.argmax(kspace_slice)
plt.xlim(peak_x - 20, peak_x + 20);

In [ ]:
W.pad

In [ ]:
plt.figure(figsize=(4, 4))
kspace_image = np.abs(get_transverse_slice(W.kmesh)) ** 2

center = np.argmax(kspace_image, axis=0)[0]
plt.imshow(kspace_image)
plt.xlim(center - 25, center + 25)
plt.ylim(center - 25, center + 25)

## Propagation

In [ ]:
drifted_w = W.drift(3.0)
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))

rspace_abs_orig = np.abs(get_transverse_slice(W.rmesh)) ** 2
rspace_abs_prop = np.abs(get_transverse_slice(drifted_w.rmesh)) ** 2
vmin = np.min((np.min(rspace_abs_orig), np.min(rspace_abs_prop)))
vmax = np.min((np.max(rspace_abs_orig), np.max(rspace_abs_prop)))
im1 = ax1.imshow(rspace_abs_orig, vmin=vmin, vmax=vmax)
im2 = ax2.imshow(rspace_abs_prop, vmin=vmin, vmax=vmax)
fig.subplots_adjust(right=1.0)
plt.colorbar(im2)
ax1.set_title("Original")
ax2.set_title("Drifted to 3m");

In [ ]:
def gaussian_func(x, a, b, c, d):
    return a * np.exp(-((x - b) ** 2) / 2.0 / c**2) + d


def gaussian_fit(xdata, ydata, initial_guess):
    popt_gaussian, pcov_gaussian = curve_fit(
        gaussian_func, xdata, ydata, p0=initial_guess
    )
    FWHM = popt_gaussian[2] * 2.355

    ydata_fit = gaussian_func(xdata, *popt_gaussian)

    spline = UnivariateSpline(xdata, ydata_fit - np.max(ydata_fit) / 2.0, s=0)
    r1, r2 = spline.roots()

    roots = [r1, r2]

    return popt_gaussian, ydata_fit, FWHM, roots

In [ ]:
zR = 2.0
X = Wavefront.gaussian_pulse(
    dims=(101, 101, 801),
    wavelength=1.35e-8,
    grid_spacing=(6e-6, 6e-6, 1e-7),
    pad=(100, 100, 40),
    nphotons=1e12,
    zR=2.0,
    sigma_z=3e-6,
)

# drift distance [m]
dz = 0.25
# drift this many steps:
num_drift_steps = 15

nx, ny = get_transverse_slice(X.rmesh).shape
wfz = np.zeros((num_drift_steps, nx, ny))
fwhmz_fit = np.zeros(num_drift_steps)

w0 = np.sqrt(zR * X.wavelength / np.pi)
z = np.linspace(
    -num_drift_steps * dz,
    num_drift_steps * dz + dz,
    2 * num_drift_steps + 1,
    endpoint=False,
)
wz = w0 * np.sqrt(1 + (z / zR) ** 2)

domain_x = X.rspace_domain[1]

X = X.drift(0.0)

for zi in range(0, num_drift_steps):
    if zi > 0:
        print("Propagating to: ", zi * dz)
        X = X.drift(dz)
    wf = np.abs(get_transverse_slice(X.rmesh)) ** 2

    popt_gaussian, ydata_fit, FWHM, roots = gaussian_fit(
        domain_x,
        np.sum(wf, axis=1),
        [8e24, 0.0, 0.0002, 0.0],
    )
    wfz[zi, :, :] = wf
    fwhmz_fit[zi] = FWHM

wfz_wx = np.sum(wfz, axis=1)
wfz_wx2 = np.vstack((np.flip(wfz_wx, axis=0), wfz_wx))

In [ ]:
plt.figure(figsize=(10, 4))

ymin, ymax = X.ranges[1]

plt.imshow(
    wfz_wx2.T,
    extent=[-num_drift_steps * dz, num_drift_steps * dz, ymin, ymax],
    aspect="auto",
    vmin=0.0,
    vmax=1.6e26,
)

plt.plot(z, wz, "-", linewidth=2, color="white")
plt.plot(z, -wz, "-", linewidth=2, color="white")

z_slice = z[int(num_drift_steps) : int(2 * num_drift_steps)]
plt.plot(z_slice, fwhmz_fit / np.sqrt(2.0 * np.log(2)), "--", linewidth=2, color="blue")
plt.plot(
    z_slice, -fwhmz_fit / np.sqrt(2.0 * np.log(2)), "--", linewidth=2, color="blue"
)

plt.plot(z_slice, fwhmz_fit / 2, "--", linewidth=2, color="yellow")
plt.plot(z_slice, -fwhmz_fit / 2, "--", linewidth=2, color="yellow")


plt.ylim(ymin, ymax)

plt.xlabel(r"z ($m$)")
plt.ylabel("y (m)")
plt.colorbar();